In [19]:
from bs4 import BeautifulSoup
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
import pandas as pd

columns = []
d = []
with open('With_Cys/pfam_1.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

data = pd.DataFrame(d, columns=columns)

with open("With_Cys/Cys_sequences_1.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

sequences = soup.sequences.findAll('sequence')

In [3]:
data

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,A0A0J6VGA8,62,156,52,157,PF13426.7,PAS_9,Domain,9,103,104,69.7,2.1e-19,1,CL0183
1,A0A0J6VGA8,175,235,173,237,PF00512.25,HisKA,Domain,3,64,67,34.0,2.2e-08,1,CL0025
2,A0A0J6VGA8,280,402,279,405,PF02518.26,HATPase_c,Domain,2,109,112,49.2,6.5e-13,1,CL0025
3,A0A0J6VGA8,427,537,427,538,PF00072.24,Response_reg,Domain,1,111,112,67.4,1.1e-18,1,CL0304
4,A0A059UGE7,331,427,324,428,PF13426.7,PAS_9,Domain,7,103,104,69.9,1.9e-19,1,CL0183
5,A0A059UGE7,633,725,624,728,PF13426.7,PAS_9,Domain,9,101,104,65.0,6.1e-18,1,CL0183
6,A0A059UGE7,813,1099,811,1099,PF00069.25,Pkinase,Domain,3,264,264,204.9,1.4e-60,1,CL0016
7,A0A098T9D3,47,149,46,150,PF13426.7,PAS_9,Domain,2,103,104,61.8,6.1e-17,1,CL0183
8,A0A098T9D3,162,308,162,311,PF00990.21,GGDEF,Domain,1,156,161,60.0,2.2e-16,1,CL0276
9,A0A098T9D3,333,568,332,568,PF00563.20,EAL,Domain,2,238,238,230.3,2.3e-68,1,CL0276


In [33]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True:
            hits = seg.find_all('hit')
            if float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15:
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos'])])

In [7]:
with open('With_Cys/final_proteins_1.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [43]:
new_seq = {}
for key in seqs.keys():
    if key in fasta:
        new_seq[key] = seqs[key]
seqs = new_seq

In [25]:
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k]
    df = seq_data.loc[seq_data['clan'] == 'CL0183']
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <=5 and hit[1] + 43 - int(df.iloc[i, 4]) <= 1:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)   

{'A0A126WWL1': [[136, 181], [400, 445]],
 'A0A0N7DQN7': [[56, 110]],
 'A0A126WX43': [[182, 227], [459, 504]],
 'A0A150KVV9': [[33, 78]],
 'A0A0G2GQ50': [[258, 303]],
 'A0A126WYI7': [[219, 264], [513, 558]],
 'A0A1B8EWL1': [[440, 496]],
 'A0A0N1L296': [[10, 55]],
 'A0A126X2S3': [[200, 245], [482, 527]],
 'A0A143GND9': [[14, 59]],
 'A0A126WZ33': [[64, 118]],
 'A0A126X4C5': [[268, 313], [554, 599]],
 'A0A061GXQ6': [[203, 248], [480, 525]],
 'A0A0L6JDT8': [[45, 90]],
 'A0A126X350': [[36, 90]],
 'A0A126X3U7': [[35, 80], [318, 363]],
 'A0A126WWR8': [[270, 315]],
 'A0A126WYW1': [[183, 228], [453, 498]],
 'A0A161IVE1': [[237, 282]],
 'A0A0S9QBJ9': [[25, 70]],
 'A0A0M6ZH06': [[30, 75]],
 'A0A126X0Q6': [[112, 157], [386, 431]],
 'A0A1C1BPE0': [[14, 59]],
 'A0A126X2Z4': [[206, 251], [483, 528]],
 'A0A126X0X9': [[112, 157]],
 'A0A126X072': [[328, 373]],
 'A0A126WUT7': [[35, 89]],
 'A0A126WYZ3': [[45, 99]],
 'A0A126X1I0': [[27, 72], [254, 299]],
 'A0A126X1S1': [[262, 307]],
 'A0A126X2L6': [[127, 17

In [46]:
new_data = data[data['seq id'].isin(seqs.keys())].reset_index(drop = True)

In [47]:
new_data

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,A0A0J6VGA8,62,156,52,157,PF13426.7,PAS_9,Domain,9,103,104,69.7,2.1e-19,1,CL0183
1,A0A0J6VGA8,175,235,173,237,PF00512.25,HisKA,Domain,3,64,67,34.0,2.2e-08,1,CL0025
2,A0A0J6VGA8,280,402,279,405,PF02518.26,HATPase_c,Domain,2,109,112,49.2,6.5e-13,1,CL0025
3,A0A0J6VGA8,427,537,427,538,PF00072.24,Response_reg,Domain,1,111,112,67.4,1.1e-18,1,CL0304
4,A0A059UGE7,331,427,324,428,PF13426.7,PAS_9,Domain,7,103,104,69.9,1.9e-19,1,CL0183
5,A0A059UGE7,633,725,624,728,PF13426.7,PAS_9,Domain,9,101,104,65.0,6.1e-18,1,CL0183
6,A0A059UGE7,813,1099,811,1099,PF00069.25,Pkinase,Domain,3,264,264,204.9,1.4e-60,1,CL0016
7,A0A098T9D3,47,149,46,150,PF13426.7,PAS_9,Domain,2,103,104,61.8,6.1e-17,1,CL0183
8,A0A098T9D3,162,308,162,311,PF00990.21,GGDEF,Domain,1,156,161,60.0,2.2e-16,1,CL0276
9,A0A098T9D3,333,568,332,568,PF00563.20,EAL,Domain,2,238,238,230.3,2.3e-68,1,CL0276


In [41]:
'A0A126WUY0' in new_data['seq id']

False